# Phase 1: Extracting Triplets from Informal Proofs

Add src to the Python Path in the Notebook

In [4]:
%load_ext autoreload
%autoreload 2
import sys
import os


# Add the project root directory to the Python path
sys.path.append(os.path.abspath(os.path.join("..")))

In [24]:
from src.phase1.extract_triplets import extract_triplets
from src.utils.neo4j_utils import Neo4JUtils
from src.utils.file_utils import read_proof

# Load LaTeX proof
proof = read_proof("data/proofs/proof1.tex")
# print(proof)

# Extract triplets
triplet = extract_triplets(proof)

# Store in Neo4J
neo4j = Neo4JUtils("bolt://localhost:7687", ("neo4j", "password"))
for entity in triplet.entities:
    neo4j.create_node(entity)
for relation in triplet.relations:
    neo4j.create_relation(relation)
neo4j.close()

In [25]:
triplet

Triplet(entities=[Entity(id='theorem_2.3_iv', name='Theorem 2.3(iv)', label='Theorem', type='Theorem'), Entity(id='matrix_A', name='Matrix A', label='Matrix', type='Matrix'), Entity(id='matrix_B', name='Matrix B', label='Matrix', type='Matrix'), Entity(id='entry_AB', name='Entry of AB', label='Entry', type='Matrix Entry'), Entity(id='entry_ATB', name='Entry of B^T A^T', label='Entry', type='Matrix Entry'), Entity(id='result', name='(A B)^{T}=B^{T} A^{T}', label='Result', type='Equation')], relations=[Relation(source='theorem_2.3_iv', target='result', type='concludes', name='concludes'), Relation(source='matrix_A', target='entry_AB', type='defines', name='defines'), Relation(source='matrix_B', target='entry_AB', type='defines', name='defines'), Relation(source='entry_AB', target='entry_ATB', type='transforms', name='transforms'), Relation(source='matrix_B', target='entry_ATB', type='transforms', name='transforms'), Relation(source='matrix_A', target='entry_ATB', type='transforms', name=